In [2]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_7.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    return max_corr if max_corr >= threshold else 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename):
    submatrix = pd.DataFrame(index=columns, columns=columns)
    for col1, col2 in product(columns, repeat=2):
        submatrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    submatrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix{filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


#filename_sub = "7_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub)

filename_cross = "7_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross)


submatrix7_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.454513,0.373279,0.180017,0.298623,0.627203,0.577719,0.471518,0.652464,0.514987,0.459465,...,0.346804,0.312829,0.34843,0.393481,0.382146,0.494089,0.526282,0.550571,0.475252,0.329992
21085.0,0.431495,0.414406,0.357717,0.282026,0.606651,0.616266,0.488607,0.656775,0.50804,0.569707,...,0.359067,0.339833,0.293642,0.417508,0.439311,0.506483,0.503646,0.536218,0.44638,0.191113
21087.0,0.47686,0.416532,0.344464,0.257498,0.532493,0.491386,0.469463,0.54838,0.41036,0.489454,...,0.393412,0.369358,0.323499,0.404392,0.389883,0.424715,0.373315,0.541189,0.445113,0.225414
21089.0,0.38886,0.313607,0.135176,0.17622,0.565711,0.571676,0.529768,0.583679,0.528953,0.442976,...,0.412305,0.322154,0.2279,0.439458,0.373088,0.505552,0.516698,0.518426,0.473587,0.38813
21091.0,0.163845,0.107113,0.116733,0.082702,0.46308,0.368147,0.306413,0.395422,0.322534,0.333657,...,0.199603,0.32125,0.235772,0.194734,0.171558,0.270179,0.477397,0.420725,0.291903,0.471585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.438537,0.503424,0.212952,0.275247,0.113465,0.230979,0.106522,0.190837,0.345965,0.263514,...,0.346948,0.201161,0.284077,0.303223,0.494742,0.380337,0.180912,0.183724,0.359355,0.364521
38091.0,0.500796,0.526605,0.382851,0.263852,0.346686,0.466604,0.36409,0.465697,0.354686,0.426523,...,0.445451,0.270752,0.325438,0.459843,0.467886,0.454521,0.335969,0.437376,0.396802,0.266598
38093.0,0.725974,0.727192,0.460716,0.370046,0.50174,0.544068,0.304591,0.634496,0.354948,0.494538,...,0.649194,0.372565,0.520676,0.616415,0.635246,0.618554,0.418026,0.526123,0.594542,0.33793
38095.0,0.391224,0.406794,0.269871,0.186652,0.258287,0.199467,0.139937,0.258466,0.085736,0.287102,...,0.324115,0.276307,0.312675,0.355835,0.318617,0.281095,0.201148,0.329377,0.20792,0.237818
